In [1]:
import torch
import pandas as pd
import altair as alt

## Load the inputs

In [2]:
words = open("names.txt", "r").read().splitlines()

This creates all training bigrams, where the data entry is the $i$-th character of a word, and the label is the $i+1$-th character,

In [3]:
training = [[xs, ys] for w in words for xs, ys in zip("."+w, w+".")]

This encodes the characters as indices in a lookup table.

In [4]:
alphabet = sorted(list(set(".".join(words))))
char_to_idx = {c: i for i, c in enumerate(alphabet)}
idx_to_char = {i: c for i, c in enumerate(alphabet)}

## Classical counting approach

Here we create a count table for every possible bigram including the special limiting character `.`.

In [5]:
counts = torch.zeros((len(alphabet), len(alphabet)), dtype=torch.int32)

In [6]:
for x, y in training:
    counts[char_to_idx[x], char_to_idx[y]] +=1

In [7]:
def plot_histogram(counts:torch.Tensor):

    df = pd.DataFrame(counts.numpy(), index=alphabet, columns=alphabet)

    # Reshape the DataFrame for Altair heatmap
    df_melted = df.melt(ignore_index=False).reset_index()
    df_melted["text"] = df_melted["index"] + df_melted["variable"]

    heatmap = alt.Chart(df_melted).mark_rect().encode(
        x=alt.X('variable:O', axis=None), #O stands for ordinal
        y=alt.Y('index:O', axis=None),
        color=alt.Color('value:Q', legend=None) #Q stands for quantitative
    ).properties(
        width=800,
        height=800
    )

    bigrams = heatmap.mark_text(baseline='bottom').encode(
        alt.Text('text:O'),
        color=alt.condition(
            alt.datum.value > 3000,
            alt.value('white'),
            alt.value('grey')
        )
    )

    c = heatmap.mark_text(baseline='top').encode(
        alt.Text('value:Q'),
        color=alt.condition(
            alt.datum.value > 3000,
            alt.value('white'),
            alt.value('grey'))
    )

    return (heatmap + bigrams + c)

In [8]:
plot_histogram(counts)

alt.LayerChart(...)

A new word is created by sampling from the row-wise multinomial distribution obtained by normalizing the count table using the row-wise sum.

### Sampling

In [9]:
g = torch.Generator().manual_seed(2147483647)
P = (counts + 1.0).float() 
P /= P.sum(1, keepdim=True)

for _ in range(5):
    generated = ""
    idx = 0
    while True:
        p = P[idx]
        idx = torch.multinomial(p, 1, replacement=True, generator=g).item()
        if idx==0:
            break
        generated += idx_to_char[idx]
    
    print(f"Multinomial: {generated}")

Multinomial: cexze
Multinomial: momasurailezitynn
Multinomial: konimittain
Multinomial: llayn
Multinomial: ka


Similarly, the same can be done using the uniform distribution.

In [10]:
for _ in range(10):
    generated = ""
    idx = 0
    while True:
        u = torch.ones(26)
        u = u / u.sum()
        idx = torch.multinomial(u, 1, replacement=True, generator=g).item()
        if idx==0:
            break
        generated += idx_to_char[idx]
    
    print(f"Uniform: {generated}")

Uniform: dbbvjhodxkduafvggx
Uniform: gmtvifayiysfxauacof
Uniform: eqovlrbxagfijqhdhwhrphqcsxbjsdqaigeaxieseavuckpbybhh
Uniform: cajkxadyaebkufutihyqnebwnmxgnxtqoadwyinwsdhrnvckgorislsbnpyhlfltlfqvqwqfakgrditfdxhucfgvuforhrmdjmxojlk
Uniform: qwhymxaqmheo
Uniform: vmkpgwoxxghkuejpamqwkmrkajabkhrt
Uniform: gmjytqicvymbxvuupucyppghgygrwdtiqggskcvstmtcetqtimqwg
Uniform: pl
Uniform: dbysyjplukxiypd
Uniform: lgqsqpsdhwlughf


### Log likelihood

In [11]:
log_p = 0.
n = 0

for ch1, ch2 in training:
    ix1 = char_to_idx[ch1]
    ix2 = char_to_idx[ch2]
    log_p += torch.log(P[ix1, ix2])
    n += 1

print(f'{log_p=}, {log_p/n}')

log_p=tensor(-559951.5625), -2.4543561935424805


## Machine Learning Approach

By Maximum Likelihood Estimation, one might want to maximize the likelihood of a word, given by the product of probabilities for each bigram in the word, i.e, $$\mathscr{L} = \prod_{i=0}^{N-1} \operatorname{Prob}(c_ic_{i+1}),$$ where $N$ is the lenght of the word, and $c_i$ is the $i$-th character of the word.
- Maximizing $\mathscr{L}$ is equivalent to maximize the log-likelihood $\log \mathscr{L}$, as $\log$ is a monotonic transformation.
- Maximizing $\log \mathscr{L}$ is equivalent to minimize $-\log \mathscr{L}$.
- Minimizing the negative log-likelihood is equivalent to minimize the average negative log-likelihhod.
- Some bigram probabilities can be $0$, so the model is smoothed by adding counts —so the logarithm is not $\infty$ when evaluated in the computer—.

In [12]:
xs = torch.tensor([char_to_idx[t[0]] for t in training])
ys = torch.tensor([char_to_idx[t[1]] for t in training])

In [13]:
xs[:5], ys[:5]

(tensor([ 0,  5, 13, 13,  1]), tensor([ 5, 13, 13,  1,  0]))

### Idea

$X W$ is the matrix of logits, so $\exp(XW)$ which applies exponentiation elementwise, can be seen as the log-counts table, equivalent to `counts` in the classical approach. This correspongs to applying the softmax function rowwise, defined by $$\frac{e^{z_i}}{\sum_{i=0}^{L} e^{z_i}},$$ where $L-1$ is the number of classes —in this case characters—. 

In [14]:
import torch.nn.functional as funct

In [15]:
x_onehot = funct.one_hot(xs, num_classes=len(alphabet)).float()
weights = torch.randn((len(alphabet), len(alphabet)), generator=g, requires_grad=True)

### Training

In [16]:
for i in range(1000):
    x_onehot = funct.one_hot(xs, num_classes=len(alphabet)).float()
    log_counts = (x_onehot @ weights).exp() #Should be similar to counts
    probs = log_counts / log_counts.sum(1, keepdim=True) #Should be similar to P
    loss = -probs[torch.arange(xs.shape[0]), ys].log().mean() + 0.001*(weights**2).mean()

    if i%100 == 0: print(f"Iter {i+1} \t|\t Loss: {loss.item():.5f}")

    weights.grad = None
    loss.backward()

    # descent = -2 * weights.grad if loss.data < 2.5 else -50 * weights.grad 
    descent = -60 * weights.grad 
    weights.data += descent 

Iter 1 	|	 Loss: 3.82622
Iter 101 	|	 Loss: 2.47115
Iter 201 	|	 Loss: 2.46328
Iter 301 	|	 Loss: 2.46098
Iter 401 	|	 Loss: 2.45994
Iter 501 	|	 Loss: 2.45939
Iter 601 	|	 Loss: 2.45905
Iter 701 	|	 Loss: 2.45884
Iter 801 	|	 Loss: 2.45869
Iter 901 	|	 Loss: 2.45859


In [17]:
loss

tensor(2.4585, grad_fn=<AddBackward0>)

### Sampling

In [18]:
g = torch.Generator().manual_seed(2147483647)

for _ in range(5):
    generated = ""
    idx = 0
    while True:
        xenc = funct.one_hot(torch.tensor([idx]), num_classes=len(alphabet)).float()
        log_counts = (xenc @ weights).exp()
        p = log_counts / log_counts.sum(1, keepdim=True) 
        idx = torch.multinomial(p, 1, replacement=True, generator=g).item()
        if idx==0:
            break
        generated += idx_to_char[idx]
    
    print(f"Neural net: {generated}")

Neural net: cexze
Neural net: momasurailezitynn
Neural net: konimittain
Neural net: llayn
Neural net: ka


Although $P$ and the trained weights matrix are not exacly the same, they are actually really close.

In [19]:
PP = weights.exp() / weights.exp().sum(1, keepdim=True)

In [28]:
torch.allclose(P, PP, atol=1e-1)

True